In [34]:
# import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import seaborn as sns
import torch
from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.datasets import MovieLens100K, MovieLens1M, LastFM
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import add_self_loops
import pickle

from uils import feature_sim_graph, feature_mask, apply_feature_mask
from data_process import process_movielens100k, process_movielens1m
from ppr_matrix import topk_ppr_matrix


In [35]:
# define contants
config = {
    'dataset': 'MovieLens100K',
    'mode': 'ics',
    'ppr_topk': 10,
    'knn_topk': 10,
    'hidden_dim': 64,
    'fea_drop_rate': 0.4,
    'fc_loss_lambda': 0.5,
    'sc_loss_lambda':0.5,
    'alpha_ppr': 0.15,
    'epsilon': 1e-4,
    'aug_user_emb_flag':0,
    'aug_item_emb_flag':0
} 

# Loading the Dataset

We split the edges of the graph using a 80/10/10 train/validation/test split.

In [36]:
def load_dataset(dataset):
    # 选择数据集
    def get_dataset(path, name):
        if name == 'MovieLens100K':
            return MovieLens100K(path)
        
        if name == 'MovieLens1M':
            return MovieLens1M(path)
        
        if name == 'Yelp':
            return LastFM(path)

    path = './data/{}/'.format(dataset)
    data = get_dataset(path, dataset)
    # print(data)
    graph = data[0]

    return graph

graph = load_dataset(config['dataset'])
print(graph)

if config['dataset'] == 'MovieLens100K':
    user_emb, item_emb, train_edge_index, val_edge_index, test_edge_index, train_ratings, val_ratings, test_ratings, user_fea_dim, item_fea_dim = process_movielens100k(
        graph, config['mode'])

if config['dataset'] == 'MovieLens1M':
    user_emb, item_emb, train_edge_index, val_edge_index, test_edge_index, train_ratings, val_ratings, test_ratings, user_fea_dim, item_fea_dim = process_movielens1m(
        graph, config['mode'])


num_users = user_emb.shape[0]
num_items = item_emb.shape[0]

print('Data load init finish!')

print(user_emb)
print(user_emb.size())
print(item_emb)
print(item_emb.size())
print(train_edge_index)
print(train_edge_index.size())
print(train_ratings)
print(train_ratings.size())

# # 绘制热力图
# row_indices = np.random.choice(item_emb.shape[0], size=18, replace=False)
# col_indices = np.random.choice(item_emb.shape[1], size=18, replace=False) # Max columns is 18
# subset_matrix = item_emb[np.ix_(row_indices, col_indices)]
# plt.figure(figsize=(18, 18)) # 根据需要调整图像大小
# sns.heatmap(subset_matrix, cmap='Blues', cbar=True, square=True)
# plt.xlabel('项目特征')
# plt.ylabel('项目')
# plt.title('矩阵可视化')
# plt.show()

HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)
Data load init finish!
tensor([[0.3288, 0.0000, 1.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.7260, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3151, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [0.2740, 0.0000, 1.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.6575, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3014, 0.0000, 1.0000,  ..., 1.0000, 0.0000, 0.0000]])
torch.Size([943, 24])
tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 

In [37]:
# 获取LLM增强后的属性emb
if (config['aug_user_emb_flag']):
    augmented_user_df = pd.read_csv('./LLM_attribute_agumentation/aug_data/{}/augmented_user_embed_matrix.csv'.format(config['dataset']), header=None)
    user_emb = torch.tensor(augmented_user_df.values, dtype=torch.float32)
    user_fea_dim = user_emb.size(1)

if (config['aug_item_emb_flag']):
    augmented_item_df = pd.read_csv('./LLM_attribute_agumentation/aug_data/{}/augmented_item_embed_matrix.csv'.format(config['dataset']), header=None)
    item_emb = torch.tensor(augmented_item_df.values, dtype=torch.float32)
    item_fea_dim = item_emb.size(1)

print(user_emb)
print(user_fea_dim)
print(user_emb.size())
print(item_emb)
print(item_fea_dim)
print(item_emb.size())

tensor([[0.3288, 0.0000, 1.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.7260, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3151, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [0.2740, 0.0000, 1.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.6575, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3014, 0.0000, 1.0000,  ..., 1.0000, 0.0000, 0.0000]])
24
torch.Size([943, 24])
tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
18
torch.Size([1682, 18])


In [38]:
def load_multiple_npz_as_tensor(npz_file):
    emb_list = []
    # 使用 numpy.load 来加载 npz 文件
    with np.load(npz_file, allow_pickle=True) as data:
        for key in data.keys():
            # 将每个键对应的嵌入数据转换为 Tensor
            emb_tensor = torch.tensor(data[key])
            emb_list.append(emb_tensor)
    
    # 将所有的嵌入张量沿着第 0 维（即行方向）拼接成一个大的二维张量
    # 这里假设每个嵌入的形状相同
    return torch.cat(emb_list, dim=0)

# 加载并转换为二维张量
user_sem_emb = load_multiple_npz_as_tensor('./LLM_semantic_agumentation/generation/emb/user_semantic_embeddings.npz').to(torch.float32)
item_sem_emb = load_multiple_npz_as_tensor('./LLM_semantic_agumentation/generation/emb/item_semantic_embeddings.npz').to(torch.float32)

def apply_pca_to_embeddings(embedding_tensor, target_dim):
    """
    对输入的嵌入张量应用PCA进行降维。

    Parameters:
    - embedding_tensor (torch.Tensor): 输入的嵌入张量
    - target_dim (int): 降维后的目标维度

    Returns:
    - torch.Tensor: 降维后的嵌入张量
    """
    # 将 PyTorch 张量转换为 NumPy 数组
    embedding_np = embedding_tensor.cpu().numpy()
    
    # 初始化 PCA，设置目标维度
    pca = PCA(n_components=target_dim)
    
    # 对数据进行 PCA 降维
    reduced_embedding_np = pca.fit_transform(embedding_np)
    
    # 将降维后的 NumPy 数组转换回 PyTorch 张量
    reduced_embedding_tensor = torch.tensor(reduced_embedding_np, dtype=torch.float32)
    
    return reduced_embedding_tensor

# 对 user_sem_emb 和 item_sem_emb 进行降维
user_sem_emb = apply_pca_to_embeddings(user_sem_emb, user_fea_dim)
item_sem_emb = apply_pca_to_embeddings(item_sem_emb, item_fea_dim)

# 输出降维后的张量形状
print(f"User embeddings tensor of shape: {user_sem_emb.shape}")
print(f"Item embeddings tensor of shape: {item_sem_emb.shape}")

User embeddings tensor of shape: torch.Size([943, 24])
Item embeddings tensor of shape: torch.Size([1682, 18])


In [39]:

# from scipy.sparse import coo_matrix

# # 提取行和列的索引
# rows = train_edge_index[0].numpy()
# cols = train_edge_index[1].numpy()
# ratings = train_ratings.numpy()

# # 构造稀疏矩阵
# sparse_matrix = coo_matrix((ratings, (rows, cols)))

# # 将稀疏矩阵转换为DataFrame
# df = pd.DataFrame({
#     'row': sparse_matrix.row,
#     'col': sparse_matrix.col,
#     'data': sparse_matrix.data
# })

# # 保存为 .csv 文件
# sparse_matrix_file_csv = "train_matrix.csv"
# df.to_csv(sparse_matrix_file_csv, index=False)

# # 从 .csv 文件加载数据
# loaded_df = pd.read_csv(sparse_matrix_file_csv)

# # 重建稀疏矩阵
# loaded_sparse_matrix = coo_matrix((loaded_df['data'], (loaded_df['row'], loaded_df['col'])))

# # 检查稀疏矩阵的大小和内容
# print("Shape of the matrix:", loaded_sparse_matrix.shape)
# print("Data array (first 10 elements):", loaded_sparse_matrix.data[:10])
# print("Row indices (first 10 elements):", loaded_sparse_matrix.row[:10])
# print("Column indices (first 10 elements):", loaded_sparse_matrix.col[:10])

# Structure Completion

In [40]:
# 1.构建属性相似的U-U图、I-I图
user_sim_edge_index = feature_sim_graph(user_emb, config['knn_topk'])
item_sim_edge_index = feature_sim_graph(item_emb, config['knn_topk'])


# 2.构建随机游走的U-I图
adj = torch.zeros((num_users + num_items, num_users + num_items), dtype=torch.float)
adj[train_edge_index[0], train_edge_index[1]] = 1
identity_matrix = torch.eye(num_users + num_items)
adj = torch.clamp((adj + identity_matrix), 0, 1) # A+I
doc_node_indices = list(range(adj.size(0)))

# 得到PPR矩阵
ppr_adj = topk_ppr_matrix(sp.csr_matrix(adj.cpu(), dtype=float), config['alpha_ppr'], config['epsilon'],
                          doc_node_indices, config['ppr_topk'], keep_nodes=doc_node_indices)
# print(ppr_adj)

# 过滤出交互矩阵的edges
rows, cols = ppr_adj.nonzero()
ppr_set = set(zip(rows, cols))
filtered_extra_edges = {(row, col) for row, col in ppr_set if 0 <= row <= num_users and num_users + 1 <= col <= num_users + num_items}

# 将过滤后的交互追加到train_edge_index
if filtered_extra_edges:
    extra_edges_tensor = torch.tensor(list(filtered_extra_edges), dtype=torch.long).t()
    edge_index = torch.cat([train_edge_index, extra_edges_tensor], dim=1)


In [41]:
# defines LightGCN model
class LightGCN_User(MessagePassing):
    """LightGCN Model as proposed in https://arxiv.org/abs/2002.02126
    """

    def __init__(self, num_users, embedding_dim, K=2, add_self_loops=False):
        """Initializes LightGCN Model

        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int, optional): Dimensionality of embeddings. Defaults to 8.
            K (int, optional): Number of message passing layers. Defaults to 3.
            add_self_loops (bool, optional): Whether to add self loops for message passing. Defaults to False.
        """
        super().__init__()
        self.num_users = num_users
        self.embedding_dim, self.K = embedding_dim, K
        self.add_self_loops = add_self_loops

        self.users_emb = nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # e_u^0
        
        nn.init.normal_(self.users_emb.weight, std=0.1)
        
    def forward(self, edge_index, user_fea):
        """Forward propagation of LightGCN Model.

        Args:
            edge_index (SparseTensor): adjacency matrix

        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """
        # 转换为SparseTensor
        sparse_edge_index = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(
            self.num_users, self.num_users))

        # compute \tilde{A}: symmetrically normalized adjacency matrix
        edge_index_norm = gcn_norm(
            sparse_edge_index, add_self_loops=self.add_self_loops)
        
        emb_0 = self.users_emb.weight # E^0 [num_users, embedding_dim]
        embs = [emb_0]
        emb_k = emb_0

        # multi-scale diffusion
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)

        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1) # E^K [num_users, embedding_dim]
        users_emb_final = emb_final # splits into e_u^K and e_i^K
        
        return users_emb_final
    
    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        # computes \tilde{A} @ x
        return matmul(adj_t, x)


In [42]:
# defines LightGCN model
class LightGCN_Item(MessagePassing):
    """LightGCN Model as proposed in https://arxiv.org/abs/2002.02126
    """

    def __init__(self, num_items, embedding_dim, K=2, add_self_loops=False):
        """Initializes LightGCN Model

        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int, optional): Dimensionality of embeddings. Defaults to 8.
            K (int, optional): Number of message passing layers. Defaults to 3.
            add_self_loops (bool, optional): Whether to add self loops for message passing. Defaults to False.
        """
        super().__init__()
        self.num_items = num_items
        self.embedding_dim, self.K = embedding_dim, K
        self.add_self_loops = add_self_loops
        
        self.items_emb = nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # e_i^0
        
        nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index, item_fea):
        """Forward propagation of LightGCN Model.

        Args:
            edge_index (SparseTensor): adjacency matrix

        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """
        # 转换为SparseTensor
        sparse_edge_index = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(
            self.num_items, self.num_items))

        # compute \tilde{A}: symmetrically normalized adjacency matrix
        edge_index_norm = gcn_norm(
            sparse_edge_index, add_self_loops=self.add_self_loops)
        
        emb_0 = self.items_emb.weight # E^0 [num_items, embedding_dim]
        embs = [emb_0]
        emb_k = emb_0

        # multi-scale diffusion
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)

        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1) # E^K [num_items, embedding_dim]
        items_emb_final = emb_final # splits into e_u^K and e_i^K
        
        return items_emb_final
    
    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        # computes \tilde{A} @ x
        return matmul(adj_t, x)

In [43]:
# defines LightGCN model
class LightGCN_Pred(MessagePassing):
    """LightGCN Model as proposed in https://arxiv.org/abs/2002.02126
    """

    def __init__(self, num_users, num_items, embedding_dim, K=2, add_self_loops=False):
        """Initializes LightGCN Model

        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int, optional): Dimensionality of embeddings. Defaults to 8.
            K (int, optional): Number of message passing layers. Defaults to 3.
            add_self_loops (bool, optional): Whether to add self loops for message passing. Defaults to False.
        """
        super().__init__()
        self.num_users, self.num_items = num_users, num_items
        self.embedding_dim, self.K = embedding_dim, K
        self.add_self_loops = add_self_loops
        
        self.mlp = nn.Linear(self.embedding_dim * 2, 1)
        nn.init.normal_(self.mlp.weight, std=0.1)

    def forward(self, edge_index, user_fea, item_fea):
        """Forward propagation of LightGCN Model.

        Args:
            edge_index (SparseTensor): adjacency matrix

        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """
        # 转换为SparseTensor
        sparse_edge_index = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(
            num_users + num_items, num_users + num_items))

        # compute \tilde{A}: symmetrically normalized adjacency matrix
        edge_index_norm = gcn_norm(
            sparse_edge_index, add_self_loops=self.add_self_loops)
        
        emb_0 = torch.cat([user_fea, item_fea]) # E^0 [num_users + num_items, embedding_dim]
        embs = [emb_0]
        emb_k = emb_0

        # multi-scale diffusion
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)

        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1) # E^K [num_users + num_items, embedding_dim]
        users_emb_final, items_emb_final = torch.split(
            emb_final, [self.num_users, self.num_items]) # splits into e_u^K and e_i^K
        
        # 节点特征分批
        users_emb = users_emb_final[edge_index[0]] # [batch_size, embedding_dim]
        items_emb = items_emb_final[edge_index[1] - self.num_users] # [batch_size, embedding_dim]

        # 为每对用户-物品连接嵌入
        combined_emb = torch.cat((users_emb, items_emb), dim = 1) # [batch_size, embedding_dim * 2]
        # 将组合的嵌入过MLP
        pred_ratings = self.mlp(combined_emb).squeeze() # [batch_size, 1]
        
        return pred_ratings
    
    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        # computes \tilde{A} @ x
        return matmul(adj_t, x)

In [44]:
class GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAE, self).__init__()
        self.encoder = GATConv(in_channels, out_channels, heads=1)
        self.decoder = GCNConv(out_channels, in_channels)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index)
        z = F.relu(self.encoder(x, edge_index))
        x_recon = self.decoder(z, edge_index)
        return z, x_recon

# Implementing FS_GNN

In [46]:
class FS_GNN(nn.Module):
    def __init__(self,
                 num_users, 
                 num_items, 
                 user_fea_dim, 
                 item_fea_dim,
                 hidden_dim):
        super(FS_GNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_users = num_users
        self.num_items = num_items

        self.transform_users = nn.Linear(user_fea_dim, self.hidden_dim)
        self.transform_items = nn.Linear(item_fea_dim, self.hidden_dim)

        self.concat_fea = nn.Linear(self.hidden_dim * 2, self.hidden_dim)
        self.gate_fc = nn.Sequential(
            nn.Linear(self.hidden_dim * 2, self.hidden_dim),
            nn.Sigmoid()
            )
        self.gate_user = nn.Sequential(
            nn.Linear(self.hidden_dim * 2, self.hidden_dim),
            nn.Sigmoid()
            )
        self.gate_item = nn.Sequential(
            nn.Linear(self.hidden_dim * 2, self.hidden_dim),
            nn.Sigmoid()
            )
        self.mlp = nn.Linear(self.hidden_dim * 2, 1)
        self.mse_loss = torch.nn.MSELoss()

        nn.init.xavier_normal_(self.transform_users.weight)
        nn.init.xavier_normal_(self.transform_items.weight)
        nn.init.xavier_normal_(self.concat_fea.weight)
        nn.init.xavier_uniform_(self.gate_fc[0].weight)
        nn.init.xavier_uniform_(self.gate_user[0].weight)
        nn.init.xavier_uniform_(self.gate_item[0].weight)
        nn.init.normal_(self.mlp.weight, std=0.1)
            
        # LightGCN layers
        self.gcn_pred = LightGCN_Pred(self.num_users, self.num_items, self.hidden_dim)
        self.gcn_user = LightGCN_User(self.num_users, self.hidden_dim)
        self.gcn_item = LightGCN_Item(self.num_items, self.hidden_dim)
        # GAE layers
        self.gae = GAE(in_channels = self.hidden_dim, out_channels = self.hidden_dim)

    def forward(self, edge_index, user_sim_edge_index, item_sim_edge_index, user_fea_emb, item_fea_emb, user_sem_emb, item_sem_emb, device):
    
        # 统一维度
        user_fea_emb = self.transform_users(user_fea_emb)
        item_fea_emb = self.transform_items(item_fea_emb)
        user_sem_emb = self.transform_users(user_sem_emb)
        item_sem_emb = self.transform_items(item_sem_emb)

        '''Feature Completion'''
        x = torch.cat([user_fea_emb, item_fea_emb])
        mask = feature_mask(x, config['fea_drop_rate'])
        apply_feature_mask(x, mask)
        z, x_recon = self.gae(x, edge_index)
        _, item_fea_gae = torch.split(z, [self.num_users, self.num_items])
        # 计算fc_loss
        fc_loss = F.mse_loss(x, x_recon) 

        # 节点属性补全
        g = self.gate_fc(torch.cat([item_fea_emb, item_fea_gae], dim=1))  
        item_fea_fc = g * item_fea_emb + (1 - g) * item_fea_gae 
        
        '''Structure Completion'''
        user_fea_sim = self.gcn_user(user_sim_edge_index, user_fea_emb)
        item_fea_sim = self.gcn_item(item_sim_edge_index, item_fea_fc)
        # 计算sc_loss
        batch_user_fea_sim = user_fea_sim[edge_index[0]] # [batch_size, embedding_dim]
        batch_item_fea_sim = item_fea_sim[edge_index[1] - self.num_users] # [batch_size, embedding_dim]
        sc_pred = self.mlp(torch.cat((batch_user_fea_sim, batch_item_fea_sim), dim = 1)).squeeze() # [batch_size, 1]
        ratings = torch.ones(batch_user_fea_sim.size(0)).to(device)
        sc_loss = self.mse_loss(ratings, sc_pred)

        # 融合U-U特征和I-I特征
        g_user = self.gate_user(torch.cat((user_fea_sim, user_fea_emb), dim=1))  
        final_user_fea_emb = g_user * user_fea_sim + (1 - g_user) * user_fea_emb 
        g_item = self.gate_item(torch.cat((item_fea_sim, item_fea_emb), dim=1))
        final_item_fea_emb = g_item * item_fea_sim + (1 - g_item) * item_fea_emb



        pred_ratings = self.gcn_pred(edge_index, final_user_fea_emb, final_item_fea_emb)


        return pred_ratings, fc_loss, sc_loss, item_fea_gae


In [47]:
# wrapper function to evaluate model
def evaluation(model, real_ratings, edge_index, user_emb, item_emb, batch_size, device):
    label_lst, pred_lst = [], []
    total_loss, total_rmse, total_mae = 0,0,0
    total_count = 0
    mse_loss = torch.nn.MSELoss()

    data_loader = torch.utils.data.DataLoader( 
        range(edge_index.size(1)),
        shuffle=True,
        batch_size = batch_size,
    )
    for index in data_loader:
        # 获取批次数据
        batch_edges = edge_index[:, index].to(device)
        batch_ratings = real_ratings[index].to(device)
        
        # get pred_matrix
        pred_ratings, fc_loss, sc_loss, item_fea_gae = model.forward(batch_edges, user_sim_edge_index, item_sim_edge_index, user_emb, item_emb, user_sem_emb, item_sem_emb, device)
        
        pred_ratings = pred_ratings.to(device)

        pred_loss = mse_loss(batch_ratings, pred_ratings)
        
        loss = pred_loss + config['fc_loss_lambda'] * fc_loss + config['sc_loss_lambda'] * sc_loss

        batch_ratings = batch_ratings.cpu().data.numpy()
        pred_ratings = pred_ratings.cpu().data.numpy()

        mae = np.sum(np.abs(pred_ratings - batch_ratings))
        rmse = np.sum((pred_ratings - batch_ratings) ** 2)
        
        total_loss += loss.item() * len(batch_ratings)
        total_rmse += rmse
        total_mae += mae
        total_count += len(batch_ratings)

    avg_loss = total_loss / total_count
    avg_rmse = np.sqrt(total_rmse / total_count)
    avg_mae = total_mae / total_count

    return avg_loss, avg_mae, avg_rmse

# Training

In [48]:
# define contants
ITERATIONS = 300
BATCH_SIZE = 2048
LR = 0.005
ITERS_PER_EVAL = 20
ITERS_PER_LR_DECAY = 20
WEIGHT_DECAY = 0.0005

In [49]:
# setup
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(f"Using device {device}.")

model = FS_GNN(num_users, num_items, user_fea_dim, item_fea_dim, config['hidden_dim'])

model = model.to(device)
model.train()

mse_loss = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay = WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

train_edge_index = train_edge_index.to(device)
val_edge_index = val_edge_index.to(device)
test_edge_index = test_edge_index.to(device)
train_ratings = train_ratings.to(device)
val_ratings = val_ratings.to(device)
test_ratings = test_ratings.to(device)
user_emb = user_emb.to(device)
item_emb = item_emb.to(device)
user_sem_emb = user_sem_emb.to(device)
item_sem_emb = item_sem_emb.to(device)
user_sim_edge_index = user_sim_edge_index.to(device)
item_sim_edge_index = item_sim_edge_index.to(device)

Using device cuda:0.


In [50]:
# training loop

train_losses = []
val_losses = []
item_fea_gae = torch.zeros((1682, 64))

for iter in range(ITERATIONS):
    
    train_loader = torch.utils.data.DataLoader(
        range(train_edge_index.size(1)),
        shuffle=True,
        batch_size=BATCH_SIZE,
    )
    for index in train_loader:
        # 获取批次数据
        batch_train_edges = train_edge_index[:, index].to(device)
        batch_ratings = train_ratings[index].to(device)

        # forward propagation
        pred_ratings, fc_loss, sc_loss, item_fea_gae= model.forward(batch_train_edges, user_sim_edge_index, item_sim_edge_index, user_emb, item_emb, user_sem_emb, item_sem_emb, device)
        # loss computation
        pred_loss = mse_loss(batch_ratings, pred_ratings)
        train_loss = pred_loss + config['fc_loss_lambda'] * fc_loss + config['sc_loss_lambda'] * sc_loss
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    if iter % ITERS_PER_EVAL == 0:
        model.eval()
        val_loss, mae, rmse = evaluation(model, val_ratings, val_edge_index, user_emb, item_emb, BATCH_SIZE, device)
        print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_mae: {round(mae, 5)}, val_rmse: {round(rmse, 5)}")
        train_losses.append(train_loss.item())
        val_losses.append(val_loss)
        model.train()

    if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
        scheduler.step()
        
# # 绘制热力图
# item_fea_gae = item_fea_gae.detach().cpu()
# row_indices = np.random.choice(item_fea_gae.shape[0], size=30, replace=False)
# col_indices = np.random.choice(item_fea_gae.shape[1], size=30, replace=False) # Max columns is 18
# subset_matrix = subset_matrix.cpu()
# subset_matrix = item_fea_gae[np.ix_(row_indices, col_indices)]
# plt.figure(figsize=(30, 30)) # 根据需要调整图像大小
# sns.heatmap(subset_matrix, cmap='Blues', cbar=True, square=True)
# plt.xlabel('项目特征')
# plt.ylabel('项目')
# plt.title('矩阵可视化')
# plt.show()

[Iteration 0/300] train_loss: 3.83797, val_loss: 4.01196, val_mae: 1.01228, val_rmse: 1.29565
[Iteration 20/300] train_loss: 3.19666, val_loss: 3.36893, val_mae: 0.87059, val_rmse: 1.06487
[Iteration 40/300] train_loss: 3.12296, val_loss: 3.31886, val_mae: 0.8523, val_rmse: 1.05323
[Iteration 60/300] train_loss: 2.99266, val_loss: 3.31204, val_mae: 0.86989, val_rmse: 1.06598
[Iteration 80/300] train_loss: 2.99682, val_loss: 3.3101, val_mae: 0.87422, val_rmse: 1.0705
[Iteration 100/300] train_loss: 2.9612, val_loss: 3.3334, val_mae: 0.88666, val_rmse: 1.08287
[Iteration 120/300] train_loss: 3.01366, val_loss: 3.28108, val_mae: 0.86104, val_rmse: 1.05949
[Iteration 140/300] train_loss: 2.9801, val_loss: 3.30736, val_mae: 0.87725, val_rmse: 1.07343
[Iteration 160/300] train_loss: 3.08638, val_loss: 3.33001, val_mae: 0.88796, val_rmse: 1.08355
[Iteration 180/300] train_loss: 3.0948, val_loss: 3.33936, val_mae: 0.89343, val_rmse: 1.08892
[Iteration 200/300] train_loss: 2.96852, val_loss: 3.

KeyboardInterrupt: 

In [ ]:
iters = [iter * ITERS_PER_EVAL for iter in range(len(train_losses))]
plt.plot(iters, train_losses, label='train')
plt.plot(iters, val_losses, label='validation')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.title('training and validation loss curves')
plt.legend()
plt.show()

In [ ]:
# evaluate on test set
model.eval()

test_loss, test_mae, test_rmse = evaluation(model, test_ratings, test_edge_index, user_emb, item_emb, BATCH_SIZE, device)

print(f"[test_loss: {round(test_loss, 5)}, test_mae: {round(test_mae, 5)}, test_rmse: {round(test_rmse, 5)}")